In [1]:
from pipeline import ClaimifyPipeline
from llm_client import LLMClient
import polars as pl
import json
import os

classification_data_path = os.path.join(os.getcwd(), "../Data-Analysis-Project-28/data_for_git/covid_classifications.jsonl")

# load with utf-8 encoding
articles = []
with open(classification_data_path, "r", encoding="utf-8") as f:
    for line in f:
        articles.append(json.loads(line))

article_data_path = os.path.join(os.getcwd(), "../Data-Analysis-Project-28/data/article_texts_with_metadata.parquet")

article_data = pl.read_parquet(article_data_path)

print(articles[0]["result"])

kategorier = set([article["result"]["huvudkategori"] for article in articles])

kategori = "Svenska strategin"
selected_articles = []
for article in articles:
    if article["result"]["huvudkategori"] == kategori:
        selected_articles.append(article)

print(len(selected_articles))

combined_data = []
for article in selected_articles:
    combined = {"classification": article["result"], "article": article_data.filter(pl.col("id") == article["id"]).to_dicts()[0]}
    combined_data.append(combined)

print(combined_data[1]["article"]["publication"])
print(combined_data[1]["article"]["body_text"])

{'resonemang': 'Artikeln rapporterar om andelen personer med antikroppar mot covid-19 i olika svenska städer, vilket handlar om befolkningens immunitet mot viruset.', 'kategorier': ['Immunitet'], 'huvudkategori': 'Immunitet', 'källor': None, 'personer': ['Henrik Forsberg'], 'länder': ['Sverige']}
322
aftonbladet
Nya boken ”Flocken” avslöjar spelet bakom den så kallade svenska strategin.

Genom att gå igenom en mängd mejl har författaren Johan Anderberg pusslat ihop födelsen av statsepidemiolog Anders Tegnells covid-19-strategi. Svenska Dagbladet har fått tag på ett bearbetat, ännu opublicerat utdrag ur boken som beskriver i detalj spelet bakom det som skulle bli Sveriges förmodligen mest debatterade vägval året 2020.
Den 6 mars fick Anders Tegnell ett varnande mejl från matematikern Tom Britton som räknat på R-värdet för viruset. Där framförde han sin åsikt att  Folkhälsomyndigheten var för passiv: ”Med så många smittade som nu finns i Sverige så kommer smittspridning att ske. Vårt sam

In [2]:
import threading
os.environ["OPENAI_API_KEY"] = ""
model = "openai/gpt-oss-120b"
llm_client = LLMClient(model)

pipeline = ClaimifyPipeline(llm_client, question="Vilka påståenden nämns om den svenska strategin för att hantera Coronapandemin?")
results = {}
result_lock = threading.Lock()


def run_pipeline(sample, result, result_lock):
    article = sample["article"]
    prompt = article["title"] + "\n" + article["actual_lead_text"] + "\n" + article["body_text"]
    result = pipeline.run(prompt)
    with result_lock:
        results[article["id"]] = result


threads = []
for i, sample in enumerate(combined_data[1:2]):
    thread = threading.Thread(target=run_pipeline, args=(sample, results, result_lock))
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

# save result to json
notebook_dir = os.getcwd()
dir_path = os.path.join(notebook_dir, "claims_results")
print(dir_path)
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
# find the current highest run number
if not os.path.exists(os.path.join(dir_path, "current_run.json")):
    with open(os.path.join(dir_path, "current_run.json"), "w") as f:
        json.dump({model: 0}, f)
with open(os.path.join(dir_path, "current_run.json"), "r") as f:
    runs_dict = json.load(f)
    if runs_dict.get(model) is None:
        run_number = 0
        runs_dict[model] = 0
    else:
        run_number = runs_dict[model]



file_path = os.path.join(dir_path, f"{model.split('/')[-1]}_{run_number}.json")
with open(file_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False)

runs_dict[model] = run_number + 1
with open(os.path.join(dir_path, "current_run.json"), "w") as f:
    json.dump(runs_dict, f)


Error validating response, retrying: 1 validation error for DekomponeringsSvar
  Invalid JSON: EOF while parsing a string at line 5 column 1854 [type=json_invalid, input_value='{  \n  "språk": "svensk...enumerate the claims.  ', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid
Parsed response: {  
  "språk": "svenska",  
  "mening": "Nej",  
  "referentiella_termer": null,  
  "maximalt_förtydligad_mening": "I utdraget framkommer flera påståenden om den svenska strategin för att hantera coronapandemin:\n\n1. **Huvudvikt på snabb respons** – \"Om Tegnell fick en liten lucka kunde de vara där inom 30 minuter, enligt SvD.\", vilket innebär att myndigheten ansåg det kritiskt att kunna agera omedelbart.\n\n2. **Involvering av externa experter** – \"Till sist fick de tre männen komma till myndigheten. Och de fick leverera sina varningar. Så under bara några timmar blev Tegnell varnad av fyra framstående personer inom området.\" – Detta antyder

In [3]:
for result in results.values():
    for claim in result:
        print(claim)

Sverige valde medvetet en strategi som byggde på naturlig smittspridning för att uppnå flockimmunitet.
Tegnell fick varningsmeddelanden från experter innan strategin implementerades.
Total nedstängning av samhället i fyra veckor
Omfattande testning och kontaktspårning med tvåveckorskarantän för alla nära kontakter
Tillåta smittspridning för att så småningom nå ”herd immunity”
Peet Tüll bedömde att strategi två – testning, spårning och karantän – var den bästa
Folkhälsomyndigheten var för passiv
Med så många smittade i Sverige kommer smittspridning att ske
Sveriges samhälle är inte särskilt annorlunda än Italiens eller Kinas, så spridning är sannolik
Tre framstående läkare ansåg att det var viktigt att träffa Anders Tegnell snabbt
Anders Tegnell fick varningar från dessa läkare
Tegnell fick varningar från tre män och totalt fyra framstående experter på kunskapsområdet.
Den 15 mars 2020 skrev Peet Tüll, tidigare chef för Socialstyrelsens smittskyddsenhet, ett mejl till Anders Tegnell där

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI


Client = OpenAI(base_url="http://localhost:4000/v1", api_key="EMPTY")
def get_embeddings(prompts):
    response = Client.embeddings.create(
        model="Qwen/Qwen3-Embedding-8B",
        input=prompts
    )
    return [item.embedding for item in response.data]

print("Embedding claims...")
all_article_claims = []
for id, text_claims in results.items():
    article_claims = []
    embeddings = get_embeddings(text_claims)
    for claim, embedding in zip(text_claims, embeddings):
        article_claims.append({"claim": claim, "id": id, "embedding": embedding})
    all_article_claims.append(article_claims)
print("Done embedding claims")

Embedding claims...
Done embedding claims


In [5]:
print(len(all_article_claims[0]))

75


In [6]:
all_articles_embedding_groups = []
for article_claims in all_article_claims:
    embedding_groups = []
    for claim in article_claims:
        if len(embedding_groups) == 0:
            embedding_groups.append({"representative": claim, "members": [claim]})
        else:
            most_similar_group = max(embedding_groups, key=lambda group: cosine_similarity([embedding], [group["representative"]["embedding"]])[0])
            if most_similar_group is not None and cosine_similarity([embedding], [most_similar_group["representative"]["embedding"]])[0] > 0.65:
                most_similar_group["members"].append(claim)
            else:
                embedding_groups.append({"representative": claim, "members": [claim]})
    all_articles_embedding_groups.append(embedding_groups)
final_claims = []
for article in all_articles_embedding_groups:
    for article_embedding_group in article:
        representative = article_embedding_group["representative"]
        representative["count"] = len(article_embedding_group["members"])
        final_claims.append(representative)


In [ ]:

for claim in final_claims:
    print(claim["claim"])


Sverige valde medvetet en strategi som byggde på naturlig smittspridning för att uppnå flockimmunitet.
Tegnell fick varningsmeddelanden från experter innan strategin implementerades.
Total nedstängning av samhället i fyra veckor
Omfattande testning och kontaktspårning med tvåveckorskarantän för alla nära kontakter
Tillåta smittspridning för att så småningom nå ”herd immunity”
Peet Tüll bedömde att strategi två – testning, spårning och karantän – var den bästa
Folkhälsomyndigheten var för passiv
Med så många smittade i Sverige kommer smittspridning att ske
Sveriges samhälle är inte särskilt annorlunda än Italiens eller Kinas, så spridning är sannolik
Tre framstående läkare ansåg att det var viktigt att träffa Anders Tegnell snabbt
Anders Tegnell fick varningar från dessa läkare
Tegnell fick varningar från tre män och totalt fyra framstående experter på kunskapsområdet.
Den 15 mars 2020 skrev Peet Tüll, tidigare chef för Socialstyrelsens smittskyddsenhet, ett mejl till Anders Tegnell där

In [8]:
data_result_path = os.path.join(os.getcwd(), "../Data-Analysis-Project-28/data_for_git/extracted_claims.json")
with open(data_result_path, "w", encoding="utf-8") as f:
    json.dump(final_claims, f, ensure_ascii=False)